# Classification: Cats VS Dogs

We downloaded the dataset from kaggle. The training archive contains 25,000 images of dogs and cats. 

# Using own data with included Datasets

In [1]:
import os
import mxnet as mx
import matplotlib.pyplot as plt
from mxnet import gluon
from mxnet import nd, autograd

In [2]:
def aug_transform(data, label):
    data = data.astype('float32')/255 # normalize the data
    size= 224
    aug = mx.image.ForceResizeAug((size,size)) # resize the image to 224x224
    data = aug(data)
    data=data.transpose((2, 0, 1))
    return data, label

In [3]:
# x = nd.random_normal(shape=(10,224,224,3))
# y = aug_transform(x,x)

In [4]:
data_folder = "data"
dataset_name = "CatsVsDogs"

training_path = os.path.join(data_folder, dataset_name)

train_dataset = mx.gluon.data.vision.datasets.ImageFolderDataset(training_path,transform=aug_transform)

In [5]:
sample_idx = 17990
sample = train_dataset[sample_idx]
data = sample[0]
label = sample[1]

#plt.imshow(data.asnumpy(), cmap='gray')
print("Data type: {}".format(data.shape))
print("Label: {}".format(label))
print("Label description: {}".format(train_dataset.synsets[label]))
assert label == 1

Data type: (3, 224, 224)
Label: 1
Label description: dog


# Batch size and DataLoader for train_data

In [6]:
batch_size = 128
train_data = gluon.data.DataLoader(train_dataset, batch_size, shuffle=True)

# The AlexNet architecture

In [7]:
alex_net = gluon.nn.Sequential()
with alex_net.name_scope():
    #  First convolutional layer
    alex_net.add(gluon.nn.Conv2D(channels=96, kernel_size=11, strides=(4,4), activation='relu'))
    alex_net.add(gluon.nn.MaxPool2D(pool_size=3, strides=2))
    #  Second convolutional layer
    alex_net.add(gluon.nn.Conv2D(channels=192, kernel_size=5, activation='relu'))
    alex_net.add(gluon.nn.MaxPool2D(pool_size=3, strides=(2,2)))
    # Third convolutional layer
    alex_net.add(gluon.nn.Conv2D(channels=384, kernel_size=3, activation='relu'))
    # Fourth convolutional layer
    alex_net.add(gluon.nn.Conv2D(channels=384, kernel_size=3, activation='relu'))
    # Fifth convolutional layer
    alex_net.add(gluon.nn.Conv2D(channels=256, kernel_size=3, activation='relu'))
    alex_net.add(gluon.nn.MaxPool2D(pool_size=3, strides=2))
    # Flatten and apply fullly connected layers
    alex_net.add(gluon.nn.Flatten())
    alex_net.add(gluon.nn.Dense(4096, activation="relu"))
    alex_net.add(gluon.nn.Dense(4096, activation="relu"))
    alex_net.add(gluon.nn.Dense(10))

# Initialize parameters

In [8]:
ctx = mx.gpu()
alex_net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

# Optimizer 

In [9]:
trainer = gluon.Trainer(alex_net.collect_params(), 'adam', {'learning_rate': .0001})

# Softmax cross-entropy loss

In [10]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

# Evaluation Loop

In [11]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for d, l in data_iterator:
        data = d.as_in_context(ctx)
        label = l.as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
        return acc.get()[1]

# Training loop

In [ ]:
epochs = 1000
smoothing_constant = .1
metric = mx.metric.Accuracy()

for e in range(epochs):
    metric.reset()
    for i, (d, l) in enumerate(train_data):
        data = d.as_in_context(ctx)
        label = l.as_in_context(ctx)
        with autograd.record():
            output = alex_net(data)
            loss = softmax_cross_entropy(output, label)
        
        loss.backward()
        trainer.step(data.shape[0])
                
        ##########################
        #  Keep a moving average of the losses
        ##########################
        metric.update([label], [output])
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0))
                       else (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss)

        
        if i % 100 == 0 and i > 0:
                name, acc = metric.get()
                print('[Epoch %d Batch %d] Loss: %s Training: %s=%f'%(e, i, moving_loss, name, acc))

        _, train_accuracy = metric.get()
        print("Epoch %s. Loss: %s, Train_acc %s\n" % (e, moving_loss, train_accuracy))


Epoch 0. Loss: 2.3018034, Train_acc 0.5078125

Epoch 0. Loss: 2.299548125267029, Train_acc 0.48828125

Epoch 0. Loss: 2.2920658373832703, Train_acc 0.4973958333333333

Epoch 0. Loss: 2.2749305770397186, Train_acc 0.5078125

Epoch 0. Loss: 2.240529877448082, Train_acc 0.4875

Epoch 0. Loss: 2.1778787337279324, Train_acc 0.48046875

Epoch 0. Loss: 2.0786148633840087, Train_acc 0.484375

Epoch 0. Loss: 1.953319626416183, Train_acc 0.48828125

Epoch 0. Loss: 1.82948418620117, Train_acc 0.4861111111111111

Epoch 0. Loss: 1.7214135780592634, Train_acc 0.49296875

Epoch 0. Loss: 1.6181722928279525, Train_acc 0.4992897727272727

Epoch 0. Loss: 1.542740450727393, Train_acc 0.4954427083333333

Epoch 0. Loss: 1.4658534643895473, Train_acc 0.49459134615384615

Epoch 0. Loss: 1.3928355381948097, Train_acc 0.49497767857142855

Epoch 0. Loss: 1.338249338215203, Train_acc 0.49583333333333335

Epoch 0. Loss: 1.2752333216776945, Train_acc 0.4921875

Epoch 0. Loss: 1.2474314331630318, Train_acc 0.4926470

Epoch 0. Loss: 0.6982272145618609, Train_acc 0.5006610576923077

Epoch 0. Loss: 0.6995081493976365, Train_acc 0.5005963740458015

Epoch 0. Loss: 0.6994662216057763, Train_acc 0.5007694128787878

Epoch 0. Loss: 0.6985311446264731, Train_acc 0.5011748120300752

Epoch 0. Loss: 0.6992211345739974, Train_acc 0.5011077425373134

Epoch 0. Loss: 0.6990769687808585, Train_acc 0.5010416666666667

Epoch 0. Loss: 0.698974023775789, Train_acc 0.5011488970588235

Epoch 0. Loss: 0.6984446381180281, Train_acc 0.5009694343065694

Epoch 0. Loss: 0.69855930220921, Train_acc 0.5004528985507246

Epoch 0. Loss: 0.6987108332977221, Train_acc 0.5001124100719424

Epoch 0. Loss: 0.698220235979455, Train_acc 0.5002232142857143

Epoch 0. Loss: 0.6976365174733674, Train_acc 0.5005540780141844

Epoch 0. Loss: 0.6971177630916008, Train_acc 0.5007702464788732

Epoch 0. Loss: 0.6968664205039831, Train_acc 0.500819493006993

Epoch 0. Loss: 0.6970046707360708, Train_acc 0.5005425347222222

Epoch 0. Loss: 0.6966802431674

Epoch 1. Loss: 0.6984852944699685, Train_acc 0.5163690476190477

Epoch 1. Loss: 0.69746470405648, Train_acc 0.5184326171875

Epoch 1. Loss: 0.697316509073882, Train_acc 0.5181490384615385

Epoch 1. Loss: 0.6973600222938436, Train_acc 0.5176373106060606

Epoch 1. Loss: 0.6971027524119476, Train_acc 0.5173740671641791

Epoch 1. Loss: 0.6964395944040292, Train_acc 0.517578125

Epoch 1. Loss: 0.6952609989429659, Train_acc 0.5182291666666666

Epoch 1. Loss: 0.6942941701458525, Train_acc 0.5180803571428572

Epoch 1. Loss: 0.694457360152599, Train_acc 0.5162852112676056

Epoch 1. Loss: 0.6939477873243661, Train_acc 0.5163845486111112

Epoch 1. Loss: 0.6935753660557662, Train_acc 0.516695205479452

Epoch 1. Loss: 0.6933442447028751, Train_acc 0.5166807432432432

Epoch 1. Loss: 0.692554452429014, Train_acc 0.516875

Epoch 1. Loss: 0.6904192254630902, Train_acc 0.5177837171052632

Epoch 1. Loss: 0.6906194167454403, Train_acc 0.5170454545454546

Epoch 1. Loss: 0.6907474884795571, Train_acc 0.5182

Epoch 1. Loss: 0.6975821581698737, Train_acc 0.5156661184210526

Epoch 1. Loss: 0.6954570287958337, Train_acc 0.5159931282722513

Epoch 1. Loss: 0.6995936698760586, Train_acc 0.515625

Epoch 1. Loss: 0.6995521843772529, Train_acc 0.5157464378238342

Epoch 1. Loss: 0.7018602246351301, Train_acc 0.5153833762886598

Epoch 1. Loss: 0.7001549203074509, Train_acc 0.5155849358974359

Epoch 1. Loss: 0.7015025257059813, Train_acc 0.51532

Epoch 2. Loss: 0.7020762359191965, Train_acc 0.453125

Epoch 2. Loss: 0.7023771341225497, Train_acc 0.48046875

Epoch 2. Loss: 0.7038523020369701, Train_acc 0.4713541666666667

Epoch 2. Loss: 0.7041801210066282, Train_acc 0.482421875

Epoch 2. Loss: 0.703405189140493, Train_acc 0.4953125

Epoch 2. Loss: 0.7031431504701267, Train_acc 0.4947916666666667

Epoch 2. Loss: 0.702195552761676, Train_acc 0.49776785714285715

Epoch 2. Loss: 0.7012004623788495, Train_acc 0.5078125

Epoch 2. Loss: 0.7002105924467354, Train_acc 0.5060763888888888

Epoch 2. Loss: 0.70025650

Epoch 2. Loss: 0.6570613463173403, Train_acc 0.5533536585365854

Epoch 2. Loss: 0.6665207933887922, Train_acc 0.5535534274193549

Epoch 2. Loss: 0.6774810664211288, Train_acc 0.553125

Epoch 2. Loss: 0.6742299657184996, Train_acc 0.5536334325396826

Epoch 2. Loss: 0.6778430874156164, Train_acc 0.5535187007874016

Epoch 2. Loss: 0.6851719619340187, Train_acc 0.55328369140625

Epoch 2. Loss: 0.687853562966569, Train_acc 0.5530523255813954

Epoch 2. Loss: 0.6865132464190576, Train_acc 0.5533052884615385

Epoch 2. Loss: 0.684998710145377, Train_acc 0.5539718511450382

Epoch 2. Loss: 0.6841179482845869, Train_acc 0.5545099431818182

Epoch 2. Loss: 0.6828942852653629, Train_acc 0.5544525375939849

Epoch 2. Loss: 0.6839275500867575, Train_acc 0.5544542910447762

Epoch 2. Loss: 0.6887805648172328, Train_acc 0.5538194444444444

Epoch 2. Loss: 0.6871696401409294, Train_acc 0.5539981617647058

Epoch 2. Loss: 0.6910078366492974, Train_acc 0.5532048357664233

Epoch 2. Loss: 0.6889780855084765, Trai

...

Epoch 178. Loss: 2.815139896893877e-09, Train_acc 1.0

Epoch 178. Loss: 4.303136844981376e-09, Train_acc 1.0

Epoch 178. Loss: 3.872823160483238e-09, Train_acc 1.0

Epoch 178. Loss: 3.4855408444349143e-09, Train_acc 1.0

Epoch 178. Loss: 3.136986759991423e-09, Train_acc 1.0

Epoch 178. Loss: 3.1026848341724637e-09, Train_acc 1.0

Epoch 178. Loss: 3.0718131009354004e-09, Train_acc 1.0

Epoch 178. Loss: 2.7646317908418603e-09, Train_acc 1.0

Epoch 178. Loss: 2.4881686117576743e-09, Train_acc 1.0

Epoch 178. Loss: 2.3324840024923397e-09, Train_acc 1.0

Epoch 178. Loss: 2.378632352423289e-09, Train_acc 1.0

Epoch 178. Loss: 2.1407691171809603e-09, Train_acc 1.0

Epoch 178. Loss: 2.2992211464912138e-09, Train_acc 1.0

Epoch 178. Loss: 2.0692990318420927e-09, Train_acc 1.0

Epoch 178. Loss: 1.955501380568316e-09, Train_acc 1.0

Epoch 178. Loss: 1.94621574633235e-09, Train_acc 1.0

Epoch 178. Loss: 1.9378586644177504e-09, Train_acc 1.0

Epoch 178. Loss: 1.837205049886408e-09, Train_acc 1.0

E

Epoch 179. Loss: 7.281806658486254e-09, Train_acc 1.0

Epoch 179. Loss: 6.553625992637629e-09, Train_acc 1.0

Epoch 179. Loss: 6.45705676050747e-09, Train_acc 1.0

Epoch 179. Loss: 5.811351084456723e-09, Train_acc 1.0

Epoch 179. Loss: 5.509612703986773e-09, Train_acc 1.0

Epoch 179. Loss: 8.404537498966641e-09, Train_acc 1.0

Epoch 179. Loss: 9.79925544124755e-09, Train_acc 1.0

Epoch 179. Loss: 8.91246214903323e-09, Train_acc 1.0

Epoch 179. Loss: 8.393744875158256e-09, Train_acc 1.0

Epoch 179. Loss: 8.20629592341974e-09, Train_acc 1.0

Epoch 179. Loss: 7.3856663310777665e-09, Train_acc 1.0

Epoch 179. Loss: 7.205893109512514e-09, Train_acc 1.0

Epoch 179. Loss: 6.485303798561263e-09, Train_acc 1.0

Epoch 179. Loss: 5.92990567061557e-09, Train_acc 1.0

Epoch 179. Loss: 5.336915103554013e-09, Train_acc 1.0

Epoch 179. Loss: 4.803223593198612e-09, Train_acc 1.0

Epoch 179. Loss: 5.719883852352182e-09, Train_acc 1.0

Epoch 179. Loss: 5.147895467116964e-09, Train_acc 1.0

Epoch 179. Los

Epoch 179. Loss: 5.467356045340226e-09, Train_acc 1.0

Epoch 179. Loss: 5.758810313870924e-09, Train_acc 1.0

Epoch 179. Loss: 5.1829292824838316e-09, Train_acc 1.0

Epoch 179. Loss: 4.757768606145881e-09, Train_acc 1.0

Epoch 179. Loss: 4.281991745531293e-09, Train_acc 1.0

Epoch 179. Loss: 7.48594244073707e-09, Train_acc 1.0

Epoch 179. Loss: 1.0648894128264175e-08, Train_acc 1.0

Epoch 179. Loss: 9.584004715437758e-09, Train_acc 1.0

Epoch 179. Loss: 8.811868747714847e-09, Train_acc 1.0

Epoch 179. Loss: 9.513928651068522e-09, Train_acc 1.0

Epoch 179. Loss: 8.56253578596167e-09, Train_acc 1.0

Epoch 179. Loss: 7.706282207365503e-09, Train_acc 1.0

Epoch 179. Loss: 6.935653986628953e-09, Train_acc 1.0

Epoch 179. Loss: 6.242088587966057e-09, Train_acc 1.0

Epoch 179. Loss: 5.711011981079884e-09, Train_acc 1.0

Epoch 179. Loss: 7.095686058036194e-09, Train_acc 1.0

Epoch 179. Loss: 6.386117452232575e-09, Train_acc 1.0

Epoch 179. Loss: 5.84063795891975e-09, Train_acc 1.0

Epoch 179. 

Epoch 180. Loss: 2.0518297643456394e-09, Train_acc 1.0

Epoch 180. Loss: 1.8466467879110755e-09, Train_acc 1.0

Epoch 180. Loss: 2.407039813540983e-09, Train_acc 1.0

Epoch 180. Loss: 3.656450530486179e-09, Train_acc 1.0

Epoch 180. Loss: 3.290805477437561e-09, Train_acc 1.0

Epoch 180. Loss: 2.961724929693805e-09, Train_acc 1.0

Epoch 180. Loss: 2.7586846886348573e-09, Train_acc 1.0

Epoch 180. Loss: 2.8553452052086424e-09, Train_acc 1.0

Epoch 180. Loss: 2.569810684687778e-09, Train_acc 1.0

Epoch 180. Loss: 3.7098125899637982e-09, Train_acc 1.0

Epoch 180. Loss: 3.431963582877851e-09, Train_acc 1.0

Epoch 180. Loss: 3.7406927603673754e-09, Train_acc 1.0

Epoch 180. Loss: 3.4597557362410705e-09, Train_acc 1.0

Epoch 180. Loss: 3.486309148054234e-09, Train_acc 1.0

Epoch 180. Loss: 4.814057091199832e-09, Train_acc 1.0

Epoch 180. Loss: 4.518915885900715e-09, Train_acc 1.0

Epoch 180. Loss: 4.0670242973106435e-09, Train_acc 1.0

Epoch 180. Loss: 5.05730448605301e-09, Train_acc 1.0

Epo

Epoch 181. Loss: 2.3491246428548078e-09, Train_acc 1.0

Epoch 181. Loss: 2.114212178569327e-09, Train_acc 1.0

Epoch 181. Loss: 1.995923212622827e-09, Train_acc 1.0

Epoch 181. Loss: 1.7963308913605444e-09, Train_acc 1.0

Epoch 181. Loss: 1.61669780222449e-09, Train_acc 1.0

Epoch 181. Loss: 1.5481602739124738e-09, Train_acc 1.0

Epoch 181. Loss: 1.3933442465212266e-09, Train_acc 1.0

Epoch 181. Loss: 1.3471420737795366e-09, Train_acc 1.0

Epoch 181. Loss: 1.3055601183120156e-09, Train_acc 1.0

Epoch 181. Loss: 1.1750041064808141e-09, Train_acc 1.0

Epoch 181. Loss: 4.223994054640738e-09, Train_acc 1.0

Epoch 181. Loss: 3.801594649176665e-09, Train_acc 1.0

Epoch 181. Loss: 3.4214351842589985e-09, Train_acc 1.0

Epoch 181. Loss: 3.0792916658330986e-09, Train_acc 1.0

Epoch 181. Loss: 2.8644947511602215e-09, Train_acc 1.0

Epoch 181. Loss: 2.7643097687628343e-09, Train_acc 1.0

Epoch 181. Loss: 2.487878791886551e-09, Train_acc 1.0

Epoch 181. Loss: 2.3322231646083287e-09, Train_acc 1.0
